In [24]:
import json
from jsonpath_ng import parse,  Fields

def convert(origin_json, query):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        result = [match.value for match in expr.find(input_data)]
        if len(result) == 1:
            return result[0]
        return result
    except Exception as e:
        return e

originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
query = "$.data.items[:].name"

convert(originJson, query)

['Product 1', 'Product 2']

In [34]:
def replace(origin_json, query, replace_pattern, replace_value):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        
        # 查找匹配项
        matches = expr.find(input_data)
        
        # 对每个匹配项进行替换
        for match in matches:
            new_value = match.value.replace(replace_pattern, replace_value)
            match.full_path.update(input_data, new_value)
        
        # 返回修改后的 JSON 字符串
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 示例用法
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
print(originJson)
query = "$.data.items[:].name"

# 执行替换操作
result = replace(originJson, query, "1", 'xxxxxx')
print(result)

{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
{"data": {"items": [{"id": 1, "name": "Product xxxxxx"}, {"id": 2, "name": "Product 2"}]}}


In [30]:
import json
from jsonpath_ng import parse

def replace(origin_json, query, replace_value):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        
        # 查找匹配项
        matches = expr.find(input_data)
        
        # 对每个匹配项进行替换
        for match in matches:
            match.full_path.update(input_data, replace_value)
        
        # 返回修改后的 JSON 字符串
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 示例用法
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
print("原始JSON:", originJson)

query = "$.data.items[:].name"

# 执行替换操作
result = replace(originJson, query, "x")
print("替换后的JSON:", result)

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
替换后的JSON: {"data": {"items": [{"id": 1, "name": "x"}, {"id": 2, "name": "x"}]}}


In [50]:
import json
from jsonpath_ng import parse

def replace_key(origin_json, query, new_key):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        
        # 查找匹配项
        matches = expr.find(input_data)
        
        # 对每个匹配项进行键名替换
        for match in matches:
            parent = match.context.value
            if isinstance(parent, dict):
                value = parent.pop('name', None)
                if value is not None:
                    parent[new_key] = value
        
        # 返回修改后的 JSON 字符串
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 示例用法
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
print("原始JSON:    ", originJson)

query = "$.data.items"

# 执行替换操作
result = replace_key(originJson, query, "x")
print("替换后的JSON:", result)

原始JSON:     {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
替换后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}


In [58]:
import json
from jsonpath_ng import parse

def replace_key(origin_json, query, new_key):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        
        # 查找匹配项
        matches = expr.find(input_data)
        
        # 对每个匹配项进行键名替换
        for match in matches:
            parent = match.context.value
            if isinstance(parent, dict):
                old_key = match.path.fields[0]
                if old_key in parent:
                    value = parent.pop(old_key)
                    parent[new_key] = value
            elif isinstance(parent, list):
                for item in parent:
                    if isinstance(item, dict) and old_key in item:
                        value = item.pop(old_key)
                        item[new_key] = value
        
        # 返回修改后的 JSON 字符串
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 测试函数
def test_replace_key(origin_json, query, new_key):
    print(f"原始JSON: {origin_json}")
    print(f"查询: {query}")
    print(f"新键名: {new_key}")
    result = replace_key(origin_json, query, new_key)
    print(f"替换后的JSON: {result}\n")

# 示例用法
json_data = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"

# 测试用例
test_replace_key(json_data, "$.data.items[*].name", "x")  # 替换所有 items 中的 name
test_replace_key(json_data, "$.data.items", "xxx")  # 替换 items 键
test_replace_key(json_data, "$.data.items[0].name", "xx")  # 只替换第一个 item 的 name
test_replace_key(json_data, "$.data", "xxxxx")  # 只替换第一个 item 的 name

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
查询: $.data.items[*].name
新键名: x
替换后的JSON: {"data": {"items": [{"id": 1, "x": "Product 1"}, {"id": 2, "x": "Product 2"}]}}

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
查询: $.data.items
新键名: xxx
替换后的JSON: {"data": {"xxx": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
查询: $.data.items[0].name
新键名: xx
替换后的JSON: {"data": {"items": [{"id": 1, "xx": "Product 1"}, {"id": 2, "name": "Product 2"}]}}

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
查询: $.data
新键名: xxxxx
替换后的JSON: {"xxxxx": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}



In [12]:
def insert(origin_json_str, pattern, query_str):
    origin_json = json.loads(origin_json_str)
    query = json.loads(query_str)

    try:
        parser = parse(pattern)
        matches = parser.find(origin_json)
        if matches:
            matches[0].value.append(query)
        else:
            parts = pattern.split(".")
            if parts[0] == "$":
                parts = parts[1:]
            current = origin_json
            for part in parts[:-1]:
                if part not in current:
                    current[part] = {}
                current = current[part]
            current[parts[-1]] = [query]
        return json.dumps(origin_json)
    except Exception as e:
        return str(e)

data_str = '{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}'
print(data_str)

query_str = '{"id": 3, "name": "Product 3"}'
print(insert(data_str, "$.data.items", query_str))

{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}, {"id": 3, "name": "Product 3"}]}}


In [7]:
originJson = "{\"url\": \"https://lilithgames.feishu.cn/wiki/JvsDwD69SixMsBkVQQJcYHRQnse\"}\n"
query = "url"

convert(originJson, query)

'https://lilithgames.feishu.cn/wiki/JvsDwD69SixMsBkVQQJcYHRQnse'

In [3]:
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
query = "data.items[:].name"

convert(originJson, query)

['Product 1', 'Product 2']

In [5]:
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
query = "$.data.items[:].name"

convert(originJson, query)

['Product 1', 'Product 2']

In [6]:
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
query = "$.data"

convert(originJson, query)

{'items': [{'id': 1, 'name': 'Product 1'}, {'id': 2, 'name': 'Product 2'}]}

In [11]:
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
query = "$.data"

convert(originJson, query)

{'items': [{'id': 1, 'name': 'Product 1'}, {'id': 2, 'name': 'Product 2'}]}

In [15]:
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
query = "data.items[0].name"

convert(originJson, query)

'Product 1'

In [ ]:
{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}

In [ ]:
{"url": "https://lilithgames.feishu.cn/wiki/GDjWwnyN0ieJJskLRSxcBglLnxb"}

In [5]:
from jsonpath_ng import parse
import json

def insert(origin_json_str, pattern, query_str):
    origin_json = json.loads(origin_json_str)
    query = json.loads(query_str)

    try:
        parser = parse(pattern)
        matches = parser.find(origin_json)
        if matches:
            matches[0].value.append(query)
        else:
            parts = pattern.split(".")
            if parts[0] == "$":
                parts = parts[1:]
            current = origin_json
            for part in parts[:-1]:
                if part not in current:
                    current[part] = {}
                current = current[part]
            current[parts[-1]] = [query]
        return json.dumps(origin_json)
    except Exception as e:
        return str(e)

data_str = '{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}'
print(data_str)

query_str = '{"id": 3, "name": "Product 3"}'
print(insert(data_str, "$.data.items", query_str))

{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}, {"id": 3, "name": "Product 3"}]}}


In [28]:
import json
from jsonpath_ng import parse

def insert(origin_json, query, new_key, new_value):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        
        # 查找匹配项
        matches = expr.find(input_data)
        
        # 对每个匹配项进行插入操作
        for match in matches:
            if isinstance(match.value, dict):
                match.value[new_key] = new_value
            else:
                # 如果匹配项不是字典，我们需要更新其父对象
                parent = match.context.value
                if isinstance(parent, dict):
                    parent[new_key] = new_value
        
        # 返回修改后的 JSON 字符串
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 示例用法
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
print("原始JSON:", originJson)
query = "$.data.items"

# 执行插入操作
result = insert(originJson, query, "category", "Electronics")
print("插入后的JSON:", result)

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
插入后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}], "category": "Electronics"}}


In [20]:
import json
from jsonpath_ng import parse

def insert(origin_json, query, new_value, index=None):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        
        # 查找匹配项
        matches = expr.find(input_data)
        
        # 对每个匹配项进行插入操作
        for match in matches:
            if isinstance(match.value, list):
                if index is None:
                    match.value.append(new_value)
                else:
                    match.value.insert(index, new_value)
            else:
                raise ValueError("The target is not an array")
        
        # 返回修改后的 JSON 字符串
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 示例用法
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
print("原始JSON:", originJson)

# 在数组末尾插入
query = "$.data.items"
result = insert(originJson, query, {"id": 3, "name": "Product 3"})
print("在末尾插入后的JSON:", result)

# 在指定位置插入
query = "$.data.items"
result = insert(originJson, query, {"id": 0, "name": "Product 0"}, index=0)
print("在开头插入后的JSON:", result)

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
在末尾插入后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}, {"id": 3, "name": "Product 3"}]}}
在开头插入后的JSON: {"data": {"items": [{"id": 0, "name": "Product 0"}, {"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}


In [41]:
import json
from jsonpath_ng import parse, jsonpath

def insert(origin_json, query, new_value, index=None, create_path=False):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        try:
            new_value = json.loads(new_value)
        except json.JSONDecodeError:
            new_value = new_value
        
        matches = expr.find(input_data)
        
        if not matches and create_path:
            # 创建新路径
            path_parts = query.strip('$').strip('.').split('.')
            current = input_data
            for part in path_parts[:-1]:
                if part not in current:
                    current[part] = {}
                current = current[part]
            current[path_parts[-1]] = new_value
        else:
            for match in matches:
                if isinstance(match.value, dict):
                    # 向对象插入键值对
                    if isinstance(new_value, dict):
                        match.value.update(new_value)
                    else:
                        raise ValueError("Cannot insert non-dict value into dict")
                elif isinstance(match.value, list):
                    # 向数组插入元素
                    if index is None:
                        match.value.append(new_value)
                    else:
                        match.value.insert(index, new_value)
                else:
                    # 替换原有值
                    match.full_path.update(input_data, new_value)
        
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 示例用法
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
print("原始JSON:", originJson)

# 向对象插入
result = insert(originJson, "$.data.items[0].name", '{\"category\": \"Electronics\"}')
print("向对象插入后的JSON:", result)

# 向数组插入
result = insert(originJson, "data.items", '{\"id\": 3, \"name\": \"Product 3\"}')
print("向数组插入后的JSON:", result)

# 创建新路径并插入
result = insert(originJson, "data.metadata.author", "John Doe", create_path=True)
print("创建新路径并插入后的JSON:", result)

# 创建新路径并插入
result = insert(originJson, "data.items[0].metadata.author", "John Doe", create_path=True)
print("创建新路径并插入后的JSON:", result)

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
向对象插入后的JSON: {"data": {"items": [{"id": 1, "name": {"category": "Electronics"}}, {"id": 2, "name": "Product 2"}]}}
向数组插入后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}, {"id": 3, "name": "Product 3"}]}}
创建新路径并插入后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}], "metadata": {"author": "John Doe"}}}
创建新路径并插入后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}], "items[0]": {"metadata": {"author": "John Doe"}}}}


In [65]:
import json
from jsonpath_ng import parse, jsonpath

def insert(origin_json, query, new_value, index=None, create_path=False):
    try:
        input_data = json.loads(origin_json)
        expr = parse(query)
        try:
            new_value = json.loads(new_value)
        except json.JSONDecodeError:
            new_value = new_value
        
        matches = expr.find(input_data)
        
        if not matches and create_path:
            # 创建新路径
            path_parts = query.strip('$').strip('.').split('.')
            current = input_data
            for i, part in enumerate(path_parts):
                if '[' in part and ']' in part:
                    # 处理数组索引
                    array_name, index = part.split('[')
                    index = int(index.rstrip(']'))
                    if array_name not in current:
                        current[array_name] = []
                    while len(current[array_name]) <= index:
                        current[array_name].append({})
                    current = current[array_name][index]
                else:
                    if i == len(path_parts) - 1:
                        current[part] = new_value
                    elif part not in current:
                        current[part] = {}
                    current = current[part]
        else:
            for match in matches:
                if isinstance(match.value, dict):
                    # 向对象插入键值对
                    if isinstance(new_value, dict):
                        match.value.update(new_value)
                    else:
                        raise ValueError("Cannot insert non-dict value into dict")
                elif isinstance(match.value, list):
                    # 向数组插入元素
                    if index is None:
                        match.value.append(new_value)
                    else:
                        match.value.insert(int(index), new_value)
                else:
                    # 替换原有值
                    match.full_path.update(input_data, new_value)
        
        return json.dumps(input_data)
    except Exception as e:
        return str(e)

# 示例用法
originJson = "{\"data\": {\"items\": [{\"id\": 1, \"name\": \"Product 1\"}, {\"id\": 2, \"name\": \"Product 2\"}]}}"
print("原始JSON:", originJson)

# 向对象插入
result = insert(originJson, "$.data.items[0].name", '{\"category\": \"Electronics\"}')
print("向对象插入后的JSON:", result)

# 向数组插入
result = insert(originJson, "data.items", '{\"id\": 3, \"name\": \"Product 3\"}', '0')
print("向数组插入后的JSON:", result)

# 创建新路径并插入
result = insert(originJson, "data.metadata.author", "John Doe", create_path=True)
print("创建新路径并插入后的JSON:", result)

# 创建新路径并插入
result = insert(originJson, "data.items[0].a_metadata.author", "John Doe", create_path=True)
print("创建新路径并插入后的JSON:", result)

原始JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
向对象插入后的JSON: {"data": {"items": [{"id": 1, "name": {"category": "Electronics"}}, {"id": 2, "name": "Product 2"}]}}
向数组插入后的JSON: {"data": {"items": [{"id": 3, "name": "Product 3"}, {"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}
创建新路径并插入后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}], "metadata": {"author": "John Doe"}}}
创建新路径并插入后的JSON: {"data": {"items": [{"id": 1, "name": "Product 1", "a_metadata": {"author": "John Doe"}}, {"id": 2, "name": "Product 2"}]}}


In [ ]:
{"data": {"items": [{"id": 1, "name": "Product 1"}, {"id": 2, "name": "Product 2"}]}}